In [6]:
import getpass
import os

api_key= os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("http://environnement.wallonie.be/de/eso/eau_distribution/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("p")
        )
    
)
docs = loader.load()
len(docs[0].page_content)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

#Embed and store the docs splits
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()


prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("quelle est la qualité de l'eau?")

C:\Users\MediaMonster\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"La qualité de l'eau est affectée par la présence de bactéries indicatrices de pollution fécale, un pH trop bas et des excès de chlore et de métaux. Des traitements de désinfection sont parfois nécessaires pour respecter les normes microbiologiques. La législation européenne impose des exigences de qualité pour l'eau potable, notamment en ce qui concerne les nitrates et la dureté de l'eau."

In [8]:
print(docs[0].page_content[:500])

 Janvier 2018
                    Cinquième révision
 Eau du robinetL’usage de l’eau à des fins alimentaires ou d’hygiène corporelle nécessite une excellente qualité physico-chimique, chimique et microbiologique.L’eau distribuée par réseaux constitue un des produits alimentaires les plus contrôlés en Région wallonne avec plus de 30.000 contrôles par an. Ces derniers sont réalisés depuis son origine jusqu'au robinet. L’eau de distribution doit répondre aux exigences de qualité imposées par la lé


In [ ]:
# Define custom prompts
from langchain.prompts import PromptTemplate

general_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Using the information provided:\n{context}\nAnswer the question:\n{question}"
)

detailed_analysis_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Based on the following detailed analysis:\n{context}\nProvide a comprehensive answer to the question:\n{question}"
)

summarized_response_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Summarize the following content:\n{context}\nAnswer briefly:\n{question}"
)

# Use a dictionary to manage different prompts
prompt_choices = {
    "general": general_prompt,
    "detailed": detailed_analysis_prompt,
    "summary": summarized_response_prompt
}

# Example: Select a specific prompt
selected_prompt = prompt_choices["summary"]

# Updated RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | selected_prompt
    | llm
    | StrOutputParser()
)

# Example question
response = rag_chain.invoke("de donde es el agua?")
print(response)


De kwaliteit van water hangt af van de oorsprong en moet voldoen aan Europese en regionale wetgeving.
